In [ ]:
## USEFUL LINK FOR CLASSES LEGEND: 

## file:///D:/Users/Bilal%20Khan/Documents/Bilal%20-%20VS%20Code%20Projects/CxC/i15_crop_mapping_2018_shp/i15_Crop_Mapping_2018_Metadata.pdf

In [4]:
# ------------- Python Imports
import numpy as np ## I wonder what this is
import pandas as pd ## I wonder what this is
import geopandas as gpd ## to deal with shapefiles
import os ## Navigate Python OS
from PIL import Image ## Python lib to work with Image importing, processing
import rasterio


# ------------- PyTorch imports
import torch ## PyTorch
import torchgeo
import torch.nn.functional as Fun
from torch.utils.data import DataLoader, Dataset ## In-house classes build to hold/iterate through data, including our image data
import torchvision ## PyTorch library containing tools specifically for Computer Vision
import torchvision.transforms as transforms ## Holds several common image transformation processing tools, to help clean data b4 training

print('done')

done


In [1]:
# -------------------------- STEP 2: Label Data -------------------------- #
SHAPEPATH1 = 'i15_crop_mapping_2018_shp\i15_Crop_Mapping_2018.shp'
SHAPEPATH2 = 'i15_crop_mapping_2019\i15_Crop_Mapping_2019.shp'

print('loading SHAPEPATH1')
shapefile2018 = gpd.read_file(SHAPEPATH1)
print('loading SHAPEPATH2')
shapefile2019 = gpd.read_file(SHAPEPATH2)

print(shapefile2018.columns)




loading SHAPEPATH1
loading SHAPEPATH2
Index(['UniqueID', 'DWR_REVISE', 'SYMB_CLASS', 'MULTIUSE', 'CLASS1',
       'SUBCLASS1', 'SPECOND1', 'IRR_TYP1PA', 'IRR_TYP1PB', 'PCNT1', 'CLASS2',
       'SUBCLASS2', 'SPECOND2', 'IRR_TYP2PA', 'IRR_TYP2PB', 'PCNT2', 'CLASS3',
       'SUBCLASS3', 'SPECOND3', 'IRR_TYP3PA', 'IRR_TYP3PB', 'PCNT3', 'CLASS4',
       'SUBCLASS4', 'SPECOND4', 'IRR_TYP4PA', 'IRR_TYP4PB', 'PCNT4', 'UCF_ATT',
       'CROPTYP1', 'ADOY1', 'CROPTYP2', 'ADOY2', 'CROPTYP3', 'ADOY3',
       'CROPTYP4', 'ADOY4', 'REGION', 'ACRES', 'COUNTY', 'LIQ_REPORT',
       'Shape_STAr', 'Shape_STLe', 'geometry'],
      dtype='object')


Since Train and Test have same size of data, we start off with using 2019 as test data, 2018 as training data

In [30]:
# -------------------------- STEP 3: Data Transformation -------------------------- #

print('\n\n 2018: \n\n')

print(shapefile2018.info(verbose='full')) ## only two columns with noticable null values
print('\n\n DESCRIBING: \n\n')
print(shapefile2018['CLASS1'].value_counts())
print('\n\n NULLS:  \n\n')
print(shapefile2018['CLASS1'].isnull().sum())
print('\n\n LIQ_REPORT NULLS: \n\n')
print(shapefile2018['LIQ_REPORT'].isnull().sum()/len(shapefile2018['LIQ_REPORT']))
print('\n\n TYPES: \n\n')
print(shapefile2018['IRR_TYP1PB'].value_counts())


print('\n\n 2019: \n\n')

print(shapefile2018.info(verbose='full')) ## only two columns with noticable null values



 2018: 


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 408290 entries, 0 to 408289
Data columns (total 44 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   UniqueID    408290 non-null  object  
 1   DWR_REVISE  23349 non-null   object  
 2   SYMB_CLASS  408290 non-null  object  
 3   MULTIUSE    408290 non-null  object  
 4   CLASS1      408290 non-null  object  
 5   SUBCLASS1   408290 non-null  object  
 6   SPECOND1    408290 non-null  object  
 7   IRR_TYP1PA  408290 non-null  object  
 8   IRR_TYP1PB  408290 non-null  object  
 9   PCNT1       408290 non-null  object  
 10  CLASS2      408290 non-null  object  
 11  SUBCLASS2   408290 non-null  object  
 12  SPECOND2    408290 non-null  object  
 13  IRR_TYP2PA  408290 non-null  object  
 14  IRR_TYP2PB  408290 non-null  object  
 15  PCNT2       408290 non-null  object  
 16  CLASS3      408290 non-null  object  
 17  SUBCLASS3   408290 non-null  object  
 18  SPEC

I am assuming ** means no crop. There are a lot of no entries, since CLASS1 refers to:

- This is the class for the first land use group on the field, and the first level of detail for land use identification. All class codes are letters, not numerals. In the statewide 2018  data set, all Main Season summer crop data begins in column CLASS2; only distinct early crops (MULTIUSE = 'D', or Mixed Use fields (MULTIUSE = 'M') will have a code in CLASS1. All agricultural class codes except for 'YP' are right justified with a leading space. All non-agricultural codes are left justified, and may have a trailing space.


**Covariates:**
- CLASSi is the i'th land identification for a patch of land. SUBCLASSi is more granular specification.
- SPECONDi is special condition, IRR_TYPiPA is irrigation status for i'th spot, RR_TYPiPB is type of irrigation system, PCNTi is percentage of area covered by the crop in the image used in ith usage.

- CLASSi, SUBCLASSi need encoding
- The two IRR Categories may be redundant

- For other categories:
    - ID is obvious and should be dropped from train/test dataset once fullData transformations are done
    - DWR_REVISE is r if polygon was revised, n if it is new polygon (irrelevant? also many nulls, jus drop column)
    - LIQ_REPORT is label? Ratio of nulls is 0.00014, very low, just remove those rows
    - SYMB_CLASS is wierd, related to CLASS, etc where i=2 (main summer harvest)
    - MULTIUSE is whether sequential or concurrent land uses are mapped within the polygon (irrelevant?)
    - UCF_ATT holds descriptions for covariates, useless



In [39]:
# -------------------------- STEP 4: Data Transformation -------------------------- #
from sklearn.preprocessing import OneHotEncoder

def lowestFreqAttributes(df, columns):
    '''
    Goal: Return an np.array, where each item is the lowest-occuring category on the nth column, formatted to resemble name after onehotencoding
    df: Pandas Dataframe
    columns: Pandas Columns
    '''
    ans = []
    for colname in columns:
        lowestFreqAtt = df[colname].value_counts().index[-1]
        ans.append(lowestFreqAtt)
        
    columnsNp = np.array(columns)
    ansNp = np.array(ans)
    
    ## Name modification to match with encoder column names
    concatStrings = np.vectorize(lambda x, y: x+'_'+y)
    
    ## These are [ans] after formatting: the final names of columns that need to be dropped after df has undergone one-hot encoding
    dropAfterOhe = concatStrings(columnsNp, ansNp)
    
    return dropAfterOhe


def oneHotEncode(df, columns, colsToDrop):
        '''
        df: dataframe requiring encoding
        columns: list of features that need to be encoded
        drop: if you want old cat coluns to be dropped from df after encoding
        colsToDrop: self-explanatory
        '''        
        ohe = OneHotEncoder(sparse_output=False)
        
        precolumns = [item for item in df.columns if item not in columns]
                
        encDf = df.reset_index(drop=True)[precolumns]
        
        for colname in columns:
            # one-hot encode
            oheRes = ohe.fit_transform(df[[colname]])
            
            # Convert to df
            ohResDf = pd.DataFrame(oheRes, columns=ohe.get_feature_names_out([colname]))
            
            # concat results with encoded df, currently empty
            encDf = pd.concat([encDf, ohResDf], axis=1)
                        
        print(f"The following {len(columns)} features were one-hot encoded: {columns}\n")
                            
        encDf = encDf.drop(columns=colsToDrop)
            
        print(f"And the following categories were dropped: {colsToDrop}\n")
        
        return encDf

    
def dataTransform(df):
    ## No modifications
    
    ## Drop ID, DWR_REVISE
    newDf = df.drop(['UniqueID', 'DWR_REVISE', 'SYMB_CLASS', 
                     'IRR_TYP1PB', 'IRR_TYP2PB', 'IRR_TYP3PB', 'IRR_TYP4PB',
                    'UCF_ATT', ], axis=1)
    
    ## I am dropping a lot of columns due to saving space, also not that relevant by definition
    
    ## Encode all other categories, drop lowest columns
    catColumns = newDf.select_dtypes(include='object').columns
    dropAfterOhe = lowestFreqAttributes(newDf, catColumns)
    finalDf = oneHotEncode(newDf, catColumns, dropAfterOhe)
    
    return finalDf
    


In [37]:
print('Printing Now...')
clean2018 = dataTransform(shapefile2018)


Printing Now...
The following 28 features were one-hot encoded: Index(['MULTIUSE', 'CLASS1', 'SUBCLASS1', 'SPECOND1', 'IRR_TYP1PA', 'PCNT1',
       'CLASS2', 'SUBCLASS2', 'SPECOND2', 'IRR_TYP2PA', 'PCNT2', 'CLASS3',
       'SUBCLASS3', 'SPECOND3', 'IRR_TYP3PA', 'PCNT3', 'CLASS4', 'SUBCLASS4',
       'SPECOND4', 'IRR_TYP4PA', 'PCNT4', 'CROPTYP1', 'CROPTYP2', 'CROPTYP3',
       'CROPTYP4', 'REGION', 'COUNTY', 'LIQ_REPORT'],
      dtype='object')

And the following categories were dropped: ['MULTIUSE_I' 'CLASS1_X' 'SUBCLASS1_3' 'SPECOND1_Y' 'IRR_TYP1PA_n'
 'PCNT1_30' 'CLASS2_I' 'SUBCLASS2_8' 'SPECOND2_B' 'IRR_TYP2PA_i'
 'PCNT2_21' 'CLASS3_R' 'SUBCLASS3_27' 'SPECOND3_A' 'IRR_TYP3PA_n'
 'PCNT3_90' 'CLASS4_YP' 'SUBCLASS4_10' 'SPECOND4_Y' 'IRR_TYP4PA_*'
 'PCNT4_42' 'CROPTYP1_C5' 'CROPTYP2_I2' 'CROPTYP3_T27' 'CROPTYP4_T10'
 'REGION_NRO' 'COUNTY_Mariposa' 'LIQ_REPORT_D15  X    **** ****']

        ADOY1  ADOY2  ADOY3  ADOY4       ACRES     Shape_STAr   Shape_STLe  \
0         0.0    0.0    0.0 

In [38]:
print(clean2018.info(verbose='full'))

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 408290 entries, 0 to 408289
Data columns (total 1215 columns):
 #     Column                          Dtype   
---    ------                          -----   
 0     ADOY1                           float64 
 1     ADOY2                           float64 
 2     ADOY3                           float64 
 3     ADOY4                           float64 
 4     ACRES                           float64 
 5     Shape_STAr                      float64 
 6     Shape_STLe                      float64 
 7     geometry                        geometry
 8     MULTIUSE_D                      float64 
 9     MULTIUSE_M                      float64 
 10    MULTIUSE_Q                      float64 
 11    MULTIUSE_S                      float64 
 12    MULTIUSE_T                      float64 
 13    CLASS1_**                       float64 
 14    CLASS1_C                        float64 
 15    CLASS1_D                        float64 
 16    CLASS1_F

In [ ]:
print(clean2018.isna().sum())

In [ ]:
# -------------------------- STEP 4: Defining CNN Class -------------------------- #
class ConvNet(nn.Module):
    def __init__(self):
        pass
    def forward(self, x):
        pass
    
# ------------- Constants Declaration
numEpochs = 4
batchSize = 4
learningRate = 0.001

model = ConvNet().to(device)
criteria = nn.CrossEntropyLoss() # Since this is a multiclass classification problem, RMSPE makes sense
optimizer = torch.optim.SGD(model.parameters(), lr=learningRate)